### Data Cleaning

In [5]:
import pandas as pd

# Load data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')



**Initial Data Overview**

In [6]:
# Concatenate train and test for unified cleaning
train['is_train'] = True
test['is_train'] = False
combined = pd.concat([train, test], sort=False).reset_index(drop=True)

# Quick structure and missing value overview
print("Shape:", combined.shape)
print("\nMissing Values:")
print(combined.isnull().sum().sort_values(ascending=False))

print("\nColumn Types:")
print(combined.dtypes)


Shape: (12970, 15)

Missing Values:
Transported     4277
CryoSleep        310
ShoppingMall     306
Cabin            299
VIP              296
Name             294
FoodCourt        289
HomePlanet       288
Spa              284
Destination      274
Age              270
VRDeck           268
RoomService      263
PassengerId        0
is_train           0
dtype: int64

Column Types:
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported      object
is_train           bool
dtype: object


In [7]:
# Add GroupID from PassengerId
combined['GroupID'] = combined['PassengerId'].str.split('_').str[0]

# Subset again now that GroupID exists
missing_hp = combined[combined['HomePlanet'].isna()].copy()

# Try to infer the group's HomePlanet using mode
group_planet = (
    combined.groupby('GroupID')['HomePlanet']
    .agg(lambda x: x.mode().iloc[0] if x.notna().sum() else None)
)

# Map that to missing_hp rows
missing_hp['ImputedGroupPlanet'] = missing_hp['GroupID'].map(group_planet)

# Show a sample
missing_hp[['PassengerId', 'GroupID', 'Cabin', 'HomePlanet', 'ImputedGroupPlanet']].head(10)


,PassengerId,GroupID,Cabin,HomePlanet,ImputedGroupPlanet
59,0064_02,0064,E/3/S,NaN,Mars
113,0119_01,0119,A/0/P,NaN,Europa
186,0210_01,0210,D/6/P,NaN,None
225,0242_01,0242,F/46/S,NaN,None
234,0251_01,0251,C/11/S,NaN,None
274,0303_01,0303,G/41/S,NaN,None
286,0315_01,0315,G/42/S,NaN,None
291,0321_01,0321,F/61/S,NaN,None
347,0382_01,0382,G/64/P,NaN,None
365,0402_01,0402,D/15/S,NaN,None


Some missing values in HomePlanet can be confidently imputed from others in the same GroupID (like 0064 -> Mars, 0119 -> Europa).

Others show up as None because that group has no known HomePlanet values to infer from.

In [8]:
# Only fill HomePlanet where group-based inference succeeded
combined.loc[combined['HomePlanet'].isna(), 'HomePlanet'] = (
    combined.loc[combined['HomePlanet'].isna(), 'GroupID'].map(group_planet)
)

# Confirm how many are still missing
remaining_missing = combined['HomePlanet'].isna().sum()
print(f"Remaining missing HomePlanet values: {remaining_missing}")

# Fill remaining missing HomePlanet values with 'Unknown'
combined['HomePlanet'] = combined['HomePlanet'].fillna('Unknown')

# Sanity check
print("HomePlanet value counts after imputation:")
print(combined['HomePlanet'].value_counts(dropna=False))

print("\nMissing Values:")
print(combined.isnull().sum().sort_values(ascending=False))



Remaining missing HomePlanet values: 157
HomePlanet value counts after imputation:
HomePlanet
Earth      6914
Europa     3175
Mars       2724
Unknown     157
Name: count, dtype: int64

Missing Values:
Transported     4277
CryoSleep        310
ShoppingMall     306
Cabin            299
VIP              296
Name             294
FoodCourt        289
Spa              284
Destination      274
Age              270
VRDeck           268
RoomService      263
HomePlanet         0
PassengerId        0
is_train           0
GroupID            0
dtype: int64


I think these next, in about this order:

- Cabin: packed with info (deck/num/side), useful for feature engineering.

- CryoSleep + VIP: Binary, easy to fill if correlated with spending.

- Spending Columns: imputing 0s for missing values if CryoSleep = True or group-based.

- Destination: Important categorical; might be inferrable from Cabin group or spending behavior.

- Age: May have skewed distribution; good for binning or group median imputation.

- Name: Low model impact, maybe drop or use surname if I want group detection.

- Transported: Target, don’t touch in test set.

Strategy for Cabin Imputation

Step 1: Extract the parts

- Even before imputing, we split it:

In [9]:
cabin_split = combined['Cabin'].str.split('/', expand=True)
combined['CabinDeck'] = cabin_split[0]
combined['CabinNum'] = pd.to_numeric(cabin_split[1], errors='coerce')
combined['CabinSide'] = cabin_split[2]

Step 2: Add missing value flags


In [10]:
combined['Cabin_missing'] = combined['Cabin'].isna().astype(int)

Step 3: Group-based Imputation
Using `GroupID` to fill missing `CabinDeck`, `CabinSide`, and even `CabinNum` if it looks clean:

In [11]:
# Fill deck/side/num from other members in the same group
deck_map = combined.groupby('GroupID')['CabinDeck'].agg(lambda x: x.mode().iloc[0] if x.notna().sum() else None)
side_map = combined.groupby('GroupID')['CabinSide'].agg(lambda x: x.mode().iloc[0] if x.notna().sum() else None)
num_map  = combined.groupby('GroupID')['CabinNum'].median()

# Apply group-based fill
mask = combined['CabinDeck'].isna()
combined.loc[mask, 'CabinDeck'] = combined.loc[mask, 'GroupID'].map(deck_map)

mask = combined['CabinSide'].isna()
combined.loc[mask, 'CabinSide'] = combined.loc[mask, 'GroupID'].map(side_map)

mask = combined['CabinNum'].isna()
combined.loc[mask, 'CabinNum'] = combined.loc[mask, 'GroupID'].map(num_map)


Step 4: Fill remaining with 'Unknown' or new category

In [12]:
combined['CabinDeck'] = combined['CabinDeck'].fillna('Unknown')
combined['CabinSide'] = combined['CabinSide'].fillna('Unknown')
combined['CabinNum'] = combined['CabinNum'].fillna(-1)  # or median, or leave as-is


Explore Cabinless Subgroup

In [14]:
no_cabin = combined[combined['Cabin_missing'] == 1]
print(no_cabin['CryoSleep'].value_counts())
print(no_cabin['RoomService'].describe())
print(no_cabin['Destination'].value_counts())

print("\nMissing Values:")
print(combined.isnull().sum().sort_values(ascending=False))


CryoSleep
False    170
True     119
Name: count, dtype: int64
count      295.000000
mean       299.684746
std       1153.231254
min          0.000000
25%          0.000000
50%          0.000000
75%         33.000000
max      14327.000000
Name: RoomService, dtype: float64
Destination
TRAPPIST-1e      204
55 Cancri e       64
PSO J318.5-22     27
Name: count, dtype: int64

Missing Values:
Transported      4277
CryoSleep         310
ShoppingMall      306
Cabin             299
VIP               296
Name              294
FoodCourt         289
Spa               284
Destination       274
Age               270
VRDeck            268
RoomService       263
PassengerId         0
HomePlanet          0
is_train            0
GroupID             0
CabinDeck           0
CabinNum            0
CabinSide           0
Cabin_missing       0
dtype: int64


    CryoSleep
    False    170
    True     119

~41% of these passengers were in CryoSleep, which is higher than the global average (~35%). Not huge, but maybe suggestive...

    RoomService Spending
    count      295.00
    mean       299.68
    std       1153.23
    min          0.00
    25%          0.00
    50%          0.00
    75%         33.00
    max      14327.00

50% of them spent zero on RoomService - that's a big deal.

Upper bound is high, 14k+, but the mean is misleading due to outliers.

Most didn’t use RoomService — strong indicator they didn’t have cabin privileges.

    Destinations
    TRAPPIST-1e      204
    55 Cancri e       64
    PSO J318.5-22     27

This roughly mirrors the full distribution where TRAPPIST-1e dominates, so destination isn’t a strong clue here.

The “Cabinless Class” Hypothesis

- Missing Cabin info clusters with low RoomService, more CryoSleep, and no spending on amenities.

- Suggests a class of passengers without cabins — possibly low-budget, cargo-hold sleepers, or shared sleep pods.

In [15]:
combined['NoAmenities'] = (
    (combined['RoomService'].fillna(0) == 0) &
    (combined['FoodCourt'].fillna(0) == 0) &
    (combined['ShoppingMall'].fillna(0) == 0) &
    (combined['Spa'].fillna(0) == 0) &
    (combined['VRDeck'].fillna(0) == 0)
).astype(int)

combined[['Cabin_missing', 'NoAmenities']].value_counts()


Cabin_missing  NoAmenities
0              0              7350
               1              5321
1              0               163
               1               136
Name: count, dtype: int64

```text
Cabin_mis  NoAmen  count
0          0       7350  : These people Had a cabin and used at least 1 amenity
0          1       5321  : Had a cabin, used No amenities
1          0       163  : No cabin, still used Some amenities (odd)
1          1       136  : No cabin and No amenities (true “cabinless class”?)
```

Insights:
- 5321 passengers had a cabin but spent nothing on amenities : cannot assume “no spending” means “no cabin”.

- Only 136 passengers are fully “Cabinless + No Amenities” : this is the strongest evidence of a low-budget, cryo-storage cohort.

Next:

- Flag those 136 rows as a LikelyBudgetPassenger

- Investigate the 163 rows that have no cabin but used amenities - might reveal data noise or shared cabins?